## 准备数据

In [77]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [78]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [79]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
    
        # 定义第一层的参数
        self.W1 = tf.Variable(tf.random.normal([784, 32]), name='W1')  # 权重矩阵的形状为[输入特征数, 输出特征数]
        self.b1 = tf.Variable(tf.zeros([32]), name='b1')  # 偏置的形状与输出特征数相同
        
        # 定义第二层的参数
        self.W2 = tf.Variable(tf.random.normal([32, 10]), name='W2')
        self.b2 = tf.Variable(tf.zeros([10]), name='b2')
        
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        
         # 将输入展平
        x =  tf.reshape(x, (tf.shape(x)[0], -1))
       
        
        # 前向传播
        x = tf.matmul(x, self.W1) + self.b1  # 第一层
        x = tf.nn.relu(x)  # ReLU激活函数
        x = tf.matmul(x, self.W2) + self.b2  # 第二层
        logits = x

        ####################
        return logits
    

        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [80]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.1*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [81]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 49.173527 ; accuracy 0.115616664
epoch 1 : loss 32.198086 ; accuracy 0.1334
epoch 2 : loss 25.40805 ; accuracy 0.16355
epoch 3 : loss 21.913671 ; accuracy 0.1893
epoch 4 : loss 19.542559 ; accuracy 0.21056667
epoch 5 : loss 17.77113 ; accuracy 0.23083334
epoch 6 : loss 16.319513 ; accuracy 0.24966666
epoch 7 : loss 15.079931 ; accuracy 0.2673
epoch 8 : loss 14.005213 ; accuracy 0.28396666
epoch 9 : loss 13.066759 ; accuracy 0.29955
epoch 10 : loss 12.241026 ; accuracy 0.31381667
epoch 11 : loss 11.51026 ; accuracy 0.3277
epoch 12 : loss 10.859932 ; accuracy 0.3411
epoch 13 : loss 10.279371 ; accuracy 0.35265
epoch 14 : loss 9.757779 ; accuracy 0.3637
epoch 15 : loss 9.286431 ; accuracy 0.3747
epoch 16 : loss 8.858755 ; accuracy 0.38451666
epoch 17 : loss 8.469978 ; accuracy 0.39438334
epoch 18 : loss 8.114214 ; accuracy 0.40303335
epoch 19 : loss 7.7881246 ; accuracy 0.4116
epoch 20 : loss 7.4884853 ; accuracy 0.4192
epoch 21 : loss 7.212893 ; accuracy 0.42641667
epoch 2